In [1]:
from platform import python_version
print(python_version()) # tested with 3.7.1
# this adds the python wrapper module directory to the path.
import os
import sys
import importlib
bindingspath = os.getcwd() + '/../build/python'
sys.path.append(bindingspath)
print(sys.path)

3.7.1
['/home/adria/Documents/Documents/TUM/dlfs/libdl/python', '/home/adria/anaconda3/lib/python37.zip', '/home/adria/anaconda3/lib/python3.7', '/home/adria/anaconda3/lib/python3.7/lib-dynload', '', '/home/adria/anaconda3/lib/python3.7/site-packages', '/home/adria/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/adria/.ipython', '/home/adria/Documents/Documents/TUM/dlfs/libdl/python/../build/python']


In [2]:
# The library wrapper
import pybindings as pyb

In [ ]:
# Load Mnist
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
mnist = fetch_openml('mnist_784', version=1, cache=True)
trX, teX, trY, teY = train_test_split(mnist.data / 255.0, mnist.target.astype("int0"), test_size = 0.33)

# Data vectors are expected in columns
trX = trX.transpose();
teX = teX.transpose();

In [ ]:
# Do one-hot encoding  of the labels
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
enc.fit(trY.reshape(-1, 1))

trLabelsOneHot = enc.transform(trY.reshape(-1, 1)).toarray()
teLabelsOneHot = enc.transform(teY.reshape(-1, 1)).toarray()

In [ ]:
# dataset properties
print("Values per Image:", trX.shape[0])
print("Number of images for training:", trX.shape[1])
print("Number of images for testing:", teX.shape[1])
print("--- ---")
print("Training Labels:", trLabelsOneHot.shape[0])
print("Training Labels:", trLabelsOneHot.shape[1])

In [ ]:
#subsample if necessary
trainSampleSize = 46900;
trXSub = trX[:,0:trainSampleSize]
trLabelsOneHotSub = trLabelsOneHot[0:trainSampleSize,:]
testSampleSize = 23100;
teXSub = teX[:,0:testSampleSize]
teLabelsOneHotSub = teLabelsOneHot[0:testSampleSize,:]
print("Values per Image:", trXSub.shape[0])
print("Number of images for training:", trXSub.shape[1])
print("Number of images for testing:", teXSub.shape[1])
print("--- ---")
print("Training Labels:", trLabelsOneHotSub.shape[0])
print("Training Labels:", trLabelsOneHotSub.shape[1])

In [ ]:
# importlib.reload(pyb) # For some reason, this appears not to be working

In [ ]:
# Set parameters of a given architecture example
exampleModel2 = pyb.CNNClassificationExampleModel()
exampleModel2.setTrainInputs(trXSub,1,28,28)
exampleModel2.setTrainLabels(trLabelsOneHotSub,10)
exampleModel2.setTestInputs(teXSub);
exampleModel2.setTestLabels(teLabelsOneHotSub)
exampleModel2.setLearningRate(0.00005)


In [ ]:
# train and test (prints some result and returns test results)
testResults = exampleModel2.runExample(2)

In [ ]:
#check test results (subset) (it seems to get 5 - 3 confused sometimes)
subset = 10;
print("true     labels:", teY[0:subset].tolist())
print("computed labels:", testResults[0:subset])


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline 
import numpy as np

# Choose a sample you want to visualize the test result for.
sampleID = 2;
sampleImage = np.reshape(teXSub[:,sampleID],(28,28))
plt.title("Predicted label for this image is %i" %testResults[sampleID])
plt.imshow(sampleImage, interpolation='nearest', cmap='gray')
plt.show()
